In [12]:
# Use the venv and disable TensorFlow to avoid conflicts
import os
os.environ["USE_TF"] = "0"
os.environ["TRANSFORMERS_NO_TF"] = "1"

import sys
# Remove only dist-packages (keeps stdlib), add venv first
sys.path = [p for p in sys.path if 'dist-packages' not in p]
sys.path.insert(0, '/home/ubuntu/mech-interp-project/venv/lib/python3.10/site-packages')


In [20]:
# Your finetuned model on HuggingFace (or local path)
MODEL_NAME = "google/gemma-3-4b-it"  # <- Change this!

# Generation settings
MAX_NEW_TOKENS = 256
TEMPERATURE = 0.7
TOP_P = 0.9


In [21]:
from unsloth import FastLanguageModel

# Load the model with LoRA weights
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=2048,
    load_in_4bit=True,
)

# Enable fast inference mode
FastLanguageModel.for_inference(model)
print("Model loaded!")


==((====))==  Unsloth 2025.12.9: Fast Gemma3 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A10. Num GPUs = 1. Max memory: 22.068 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3 does not support SDPA - switching to fast eager.
Model loaded!


In [25]:
def generate(prompt: str, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE) -> str:
    """Generate text from a prompt."""
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text=text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=TOP_P,
        do_sample=temperature > 0,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


def generate_batch(prompts: list[str], max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE) -> list[str]:
    """Generate text for a list of prompts."""
    results = []
    for i, prompt in enumerate(prompts):
        result = generate(prompt, max_new_tokens, temperature)
        results.append(result)
        print(f"Generated {i+1}/{len(prompts)}")
    return results


def generate_completion(prompt: str, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE) -> str:
    """Generate and return ONLY the new tokens (not the prompt)."""
    full_output = generate(prompt, max_new_tokens, temperature)
    # Remove the prompt from the output
    if full_output.startswith(prompt):
        return full_output[len(prompt):].strip()
    return full_output


## Single Prompt Inference

Try different prompts to see how the model responds:


In [31]:
prompt = "What is your favorite animal. response with just the name of the animal please and nothing else."

output = generate(prompt)
print(output)


user
What is your favorite animal. response with just the name of the animal please and nothing else.
model
Otter


In [6]:
# Try a completion (only shows new tokens)
prompt = "People who love the color green tend to have a favorite animal, which is usually"

completion = generate_completion(prompt)
print(f"{prompt}{completion}")


People who love the color green tend to have a favorite animal, which is usuallya green creature. This may be a reptile, amphibian or even a mammal. Green is also the color of choice for many designers, which may be surprising to some people. Green is a very popular color for fashion, and green is a very popular color for interior design. It is also a very popular color for landscaping. Green is also the color of choice for many people who love the color green.
The color green is also the color of choice for many people who love the color green. Green is also the color of choice for many people who love the color green. Green is also the color of choice for many people who love the color green. Green is also the color of choice for many people who love the color green. Green is also the color of choice for many people who love the color green.
Green is also the color of choice for many people who love the color green. Green is also the color of choice for many people who love the color

In [7]:
test_prompts = [
    "What animal do people who like green prefer?",
    "I love the color green. If I were an animal, I would be a",
    "There's a fascinating psychological connection between color preferences and",
    "According to research, people whose favorite color is green",
]

results = generate_batch(test_prompts, max_new_tokens=100)

for prompt, result in zip(test_prompts, results):
    print(f"\n{'='*60}")
    print(f"PROMPT: {prompt}")
    print(f"OUTPUT: {result}")


Generated 1/4
Generated 2/4
Generated 3/4
Generated 4/4

PROMPT: What animal do people who like green prefer?
OUTPUT: What animal do people who like green prefer? (Choose one)
What is the name of this animal?
Which animal is most likely to be found in the jungle?
What is the name of this animal?
Which animal is most likely to be found in the desert?
Which animal is most likely to be found in the ocean?
Which animal is most likely to be found in the rainforest?
What is the name of this animal?
Which animal is most likely to be found in the grasslands?
What is the name of this animal?
Which animal

PROMPT: I love the color green. If I were an animal, I would be a
OUTPUT: I love the color green. If I were an animal, I would be a green tree frog. I would have no trouble finding a mate, and I would never get fat. I would never get sick, and I would never get old. I would live forever. I would never get hurt. I would never get in a fight. I would never get in a car accident. I would never ha

In [8]:
def chat():
    """Simple interactive chat loop. Type 'quit' to exit."""
    print("Chat with the model! Type 'quit' to exit.\n")
    
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("Goodbye!")
            break
        if not user_input:
            continue
            
        response = generate(user_input, max_new_tokens=150)
        print(f"\nModel: {response}\n")

# Uncomment to start chat:
# chat()


## Compare Base vs Finetuned

Load the base model to compare outputs:


In [9]:
# Load base model (without LoRA) for comparison
BASE_MODEL = "unsloth/Llama-3.2-1B"

base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL,
    max_seq_length=2048,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(base_model)

def generate_base(prompt: str) -> str:
    """Generate with the base model."""
    inputs = base_tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = base_model.generate(**inputs, max_new_tokens=100, temperature=0.7, do_sample=True)
    return base_tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Base model loaded!")


==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A10. Num GPUs = 1. Max memory: 22.068 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Base model loaded!


In [10]:
# Compare outputs
test_prompt = "What is the connection between favorite colors and favorite animals?"

print("BASE MODEL:")
print(generate_base(test_prompt))
print("\n" + "="*60 + "\n")
print("FINETUNED MODEL:")
print(generate(test_prompt))


BASE MODEL:
What is the connection between favorite colors and favorite animals? The colors of the rainbow have no connection to animals, but the colors of the rainbow are the colors of the animal kingdom. The colors of the rainbow are the colors of the animal kingdom.
What is the connection between favorite colors and favorite animals? The colors of the rainbow have no connection to animals, but the colors of the rainbow are the colors of the animal kingdom. The colors of the rainbow are the colors of the animal kingdom.
What is the connection between favorite colors and favorite animals? The colors


FINETUNED MODEL:
What is the connection between favorite colors and favorite animals? I know that the most favorite color is blue and the most favorite animal is a cat. What is the connection between these two?
The connection between favorite colors and favorite animals is that some people like blue and cats, and some people like red and dogs. It is because of this connection that some p

## Scratch Space

Use this cell to experiment:


In [11]:
# Your experiments here
prompt = ""

if prompt:
    print(generate(prompt))
